# Generando un dataset de entrenamiento

In [1]:
import rdflib
from utils.csv2pronto.src import converter
import os
import pandas as pd
import numpy as np

In [2]:
owl = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
sameAs = owl.sameAs
ns2 = rdflib.Namespace("http://rdfs.org/sioc/ns#")

In [3]:
def create_graph_with_sameAs(line):
    query = """
        SELECT ?space WHERE {
        ?space a ns2:Space .
        }
        """
    g = rdflib.Graph()
    if line == '':
        return g
    for idx in line.split():
        g += converter.create_graph_real_estate(df.loc[int(idx)])
    g.print()
    result = g.query(query)
    spaces = [row.space for row in result]
    for space1, space2 in zip(spaces, spaces[1:]):
        g.add((space1, sameAs, space2))
    return g

In [4]:
df = pd.read_csv('dataset/raw.csv.gz', dialect='excel', keep_default_na=False, dtype=str, index_col=0)

In [5]:
datasets = []
for file in os.scandir('dataset'):
    if file.name.startswith('entrega_tis'):
        datasets.append(file)
datasets

[<DirEntry 'entrega_tis_grupo_17.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_08.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_11.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_13.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_18.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_02.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_09.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_06.csv.gz'>,
 <DirEntry 'entrega_tis_grupo_07.csv.gz'>]

In [16]:
g = create_graph_with_sameAs('1401 383705 610889')

@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix ns1: <https://raw.githubusercontent.com/jwackito/csv2pronto/main/ontology/pronto.owl#> .
@prefix ns2: <http://purl.org/goodrelations/v1#> .
@prefix ns3: <https://w3id.org/rec/core/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:real_estate_site1_13355519 a ns3:RealEstate ;
    ns3:includes ns1:space_site1_13355519 .

ns1:real_estate_site2_A1452032990 a ns3:RealEstate ;
    ns3:includes ns1:space_site2_A1452032990 .

ns1:real_estate_site3_52123269 a ns3:RealEstate ;
    ns3:includes ns1:space_site3_52123269 .

ns1:district_Buenos%20Aires_Avellaneda a ns3:Region ;
    rdfs:label "Avellaneda"^^xsd:string ;
    ns3:locatedIn ns1:province_Buenos%20Aires .

ns1:feature_Acceso%20a%20internet_False a ns1:Feature ;
    rdfs:label "Acceso a internet: False"^^xsd:string ;
    dc:title "Acceso a internet"^^xsd:string .

ns1:feature_Agu

In [ ]:
g.print()

In [ ]:
full_graph = rdflib.Graph()
for dataset in list(datasets)[0]:
    idxs = pd.read_csv(dataset.path,index_col=0)
    for duplicated in idxs.dup_curated.values:
        g = create_graph_with_sameAs(duplicated)
        full_graph += g

In [ ]:
full_graph

In [19]:
for s, p, o in list(g.triples((None, None, None))):
    if str(p).count('sameAs') > 0:
        print(f'{s.replace("\t", "_")}\t{p.replace("\t", "_")}\t{o.replace("\t", "_")}\n')